In [20]:
import os
import json
import zipfile

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [21]:
EXTRACT_PATH = "eeg_data"
ZIP_PATH = "eye-state-classification-eeg-dataset.zip"
DATA_FILE = "EEG_Eye_State_Classification.csv"

os.makedirs(EXTRACT_PATH, exist_ok=True)

if not os.path.exists(f"{EXTRACT_PATH}\{DATA_FILE}"):
    with open('kaggle.json', "r") as file:
        config = json.load(file)

    os.environ['KAGGLE_USERNAME'] = config["username"]
    os.environ['KAGGLE_KEY'] = config["key"]

    import kaggle

    !kaggle datasets download -d robikscube/eye-state-classification-eeg-dataset
    with zipfile.ZipFile(ZIP_PATH, "r") as zip_ref:
        zip_ref.extractall(EXTRACT_PATH)
else:
    print("Data exists locally, will not re-download")

Dataset URL: https://www.kaggle.com/datasets/robikscube/eye-state-classification-eeg-dataset
License(s): CC0-1.0




  0%|          | 0.00/827k [00:00<?, ?B/s]
100%|██████████| 827k/827k [00:00<?, ?B/s]
